In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

import imutils
from imutils.video import VideoStream


# Begin  Detection

* Load a pre-trained face detection model to extract faces from images.
* Run our mask detector to detect masks for each of the faces

Here, we will be trying 2 different pre-trained face detector


In [2]:
imgSize = 224

In [3]:
#Face Detector Type 3
"""
OpenCV’s deep neural network module with Caffe models, is based on the Single Shot Detector (SSD) framework 
with a ResNet base network

It requires two sets of files:

The .prototxt file(s) which define the model architecture (i.e., the layers themselves)
The .caffemodel file which contains the weights for the actual layers
"""


# For videos, this model is much faster at detecting faces, when compared to Cascade Classifiers
prototxtPath = "face_detector/deploy.prototxt"
weightsPath = "face_detector/res10_300x300_ssd_iter_140000.caffemodel"
faceDetector_type3 = cv2.dnn.readNet(prototxtPath, weightsPath)



In [4]:
#Load Mask Coverage Model
maskDetector = load_model('modelAttributes/maskCoverageModel_v2.h5')
maskDetector.load_weights("modelAttributes/weights_maskcoverage_v2.best.hdf5")

mask_label = {0:'covered',1:'uncovered'}
color_label = {0:(0,255,0),1:(255,0,0)}

In [5]:
# For plotting Bounding Box and Labels

def plotBox(frame,locs,preds):
    for (box, pred) in zip(locs, preds):
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        # determine the class label and color we'll use to draw
        # the bounding box and text
        index = pred.argmax()
        label = "{}: {:.2f}%".format(mask_label[index], max(mask, withoutMask) * 100)
        color = color_label[index]


        # display the label and bounding box rectangle on the output
        # frame
        cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # show the output frame
    return frame

In [6]:
#For Bounding box extraction and mask detection prediction

def detectMask(frame, faceNet, maskNet,conf=0.5):
    # grab the dimensions of the frame and then construct a blob
    # from it
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (400, 400),
        (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)

    # initialize our list of faces, their corresponding locations,
    # and the list of predictions from our face mask network
    faces = []
    locs = []
    preds = []

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the detection
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the confidence is
        # greater than the minimum confidence
        if confidence > conf:
            # compute the (x, y)-coordinates of the bounding box for
            # the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # ensure the bounding boxes fall within the dimensions of
            # the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            if(startX < endX and startY < endY):
                # extract the face ROI, convert it from BGR to RGB channel
                # ordering, resize it to 224x224, and preprocess it
                face = frame[startY:endY, startX:endX]
                face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                face = cv2.resize(face, (imgSize, imgSize))
                #face = face/255.0
                #face = img_to_array(face)
                face = preprocess_input(face)

                # add the face and bounding boxes to their respective
                # lists
                faces.append(face)
                locs.append((startX, startY, endX, endY))

    # only make a predictions if at least one face was detected
    if len(faces) > 0:
        # for faster inference we'll make batch predictions on *all*
        # faces at the same time rather than one-by-one predictions
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)

    # return a 2-tuple of the face locations and their corresponding
    # locations
    return plotBox(frame,locs,preds)

In [7]:
#Video Stream

import IPython
import time
import io
from PIL import Image  #Requires pillow

d = IPython.display.display("", display_id=1)

#Use 'jpeg' instead of 'png' (~5 times faster)
def array_to_image(a, fmt='jpeg'):
    #Create binary stream object
    f = io.BytesIO()
    #Convert array to binary stream object
    Image.fromarray(a).save(f, fmt)
    return IPython.display.Image(data=f.getvalue())

#Start Camera
cam = cv2.VideoCapture(0)

while True:
    try:
        ret, frame = cam.read()
        frame = cv2.flip(frame, 1)
        frame = imutils.resize(frame, width=400)
        new_frame = detectMask(frame, faceDetector_type3, maskDetector,0.6)
        new_frame = cv2.cvtColor(new_frame, cv2.COLOR_BGR2RGB)
        im = array_to_image(new_frame)
        d.update(im)
    except KeyboardInterrupt:
        print()
        cam.release()
        IPython.display.clear_output()
        print ("Stream stopped")
        break

Stream stopped
